# Asset History Generator

### Imports

In [1]:
from systemlink.messagebus.message_service import MessageService
from systemlink.messagebus.message_service_builder import MessageServiceBuilder
from systemlink.messagebus.generic_message import GenericMessage
from systemlink.messagebus.message_header import MessageHeader
import json
import uuid
from datetime import datetime, timezone
from datetime import timedelta, date
import random

### Create an AMQP connection

In [2]:
message_service_builder = MessageServiceBuilder('AssetHistoryGenerator')
message_service = MessageService(message_service_builder)

### Define helper functions

In [3]:
def start_utilization(system_controller, minion_id, assets, utilization_category='Test', task_name='Testing', username=None, start_date=datetime.utcnow().isoformat()):
    #print('Starting utilization...')
    header = MessageHeader(
        message_name='AssetPerformanceManagementRecordAssetUtilizationStartRequest',
        content_type='application/json',
        routing_param='AssetPerformanceManagement'
    )
    body = {
        "startUtilizationRecord":{
            "utilizationId":str(uuid.uuid4()),
            "minionId":minion_id,
            "systemControllerIdentification":system_controller,
            "assetIdentifications":assets,
            "utilizationCategory":utilization_category,
            "taskName":task_name,
            "userName":username,
            "utilizationTimestamp":start_date
        }
    }

    string_body = json.dumps(body)

    generic_message = GenericMessage(header=header, body=string_body)
    response = message_service.publish_synchronous_message(generic_message)
    response_json = json.loads(response.body_bytes)
    #display(response_json)
    
    return body['startUtilizationRecord']['utilizationId']

In [4]:
def end_utilization(utilization_id, end_date):
    #print('Ending utilization...')
    header = MessageHeader(
        message_name='AssetPerformanceManagementRecordAssetUtilizationEndRequest',
        content_type='application/json',
        routing_param='AssetPerformanceManagement'
    )
    body = {
        "utilizationId":utilization_id,
        "utilizationTimestamp":end_date
    }

    string_body = json.dumps(body)

    generic_message = GenericMessage(header=header, body=string_body)
    response = message_service.publish_synchronous_message(generic_message)
    response_json = json.loads(response.body_bytes)
    #display(response_json)
    return response_json

In [5]:
def query_assets():
    header = MessageHeader(
        message_name='AssetPerformanceManagementQueryAssetsRequest',
        content_type='application/json',
        routing_param='AssetPerformanceManagement'
    )
    body = {
    }

    string_body = json.dumps(body)

    generic_message = GenericMessage(header=header, body=string_body)
    response = message_service.publish_synchronous_message(generic_message)
    response_json = json.loads(response.body_bytes)
    return response_json['assets']

In [6]:
def get_system_controller_for_asset(asset, all_assets):
    system_name = asset['location']['systemName']
    minion_id = asset['location']['minionId']    
    for asset_element in all_assets:
        if asset_element['name'] == system_name:
            system_controller = {
                "modelName":asset_element['modelName'],
                "modelNumber":asset_element['modelNumber'],
                "serialNumber":asset_element['serialNumber'],
                "vendorName":asset_element['vendorName'],
                "vendorNumber":asset_element['vendorNumber'],
                "busType":asset_element['busType']
            }
            #display(system_controller)
            return minion_id, system_controller
    return None

### Configure the asset utilization generator

In [7]:
shifts = [
    {
        'start_hour':9,
        'end_hour':16,
        'operators': {
            'mvaterla': {'low_utilization_percent':0.60, 'high_utilization_percent': 0.95},
            'alweaver': {'low_utilization_percent':0.82, 'high_utilization_percent': 0.89},
            'jprewitt': {'low_utilization_percent':0.75, 'high_utilization_percent': 0.88}
        }
    },
    {
        'start_hour':17, 
        'end_hour':23,
        'operators': {
            'maxb': {'low_utilization_percent':0.21, 'high_utilization_percent': 0.90},
            'hmodi': {'low_utilization_percent':0.72, 'high_utilization_percent': 0.86},
        }
    },
    {
        'start_hour':0,
        'end_hour':8,
        'operators': {
            'cwaterma': {'low_utilization_percent':0.39, 'high_utilization_percent': 0.99},
            'skizunov': {'low_utilization_percent':0.54, 'high_utilization_percent': 0.70},
        }
    }
]

utilization_categories = ['Test', 'Debugging', 'Calibration']
task_names = ['Testing', 'Calibrating', 'Set Up']

#"2019-09-11T00:43:36.530059Z"
start_date = datetime.strptime('1/1/2019 12:01 AM', '%m/%d/%Y %I:%M %p')
end_date = datetime.strptime('9/13/2019 12:59 PM', '%m/%d/%Y %I:%M %p')

date_ranges = []
start_range = start_date
while start_range < end_date:
    end_range = datetime(start_range.year, start_range.month, start_range.day) + timedelta(days=1)
    if end_range > end_date:
        end_range = end_date  

    date_ranges.append({'start': start_range, 'end': end_range})
    start_range = start_range + timedelta(days=1)
#display(date_ranges)

### Query assets

In [12]:
assets = query_assets()
systems = []
for asset in assets:
    #display(asset)
    if asset['busType'] == 'BUILT_IN_SYSTEM' and asset['location']['parent'] == '':
        systems.append(
            {
                'system': asset,
                'children': []
            }
        )

### Assign assets to each operator

In [9]:
for shift in shifts:
    for operator in shift['operators']:
        if 'assets' not in shift['operators'][operator]:
            shift['operators'][operator]['assets'] = []

    for asset in assets:
        operators = [operator for operator in shift['operators']]
        operator = random.choice(operators)
        shift['operators'][operator]['assets'].append(asset)

### Build system/asset hierarchy

In [10]:
for asset in assets:
    #print('Asset {} with system {}'.format(asset['name'], asset['location']['systemName']))
    if asset['busType'] != 'BUILT_IN_SYSTEM' and asset['location']['systemName'] != '':
        for system in systems:
            #print('{} ?= {}'.format(asset['location']['systemName'], system['system']['name']))
            if asset['location']['systemName'] == system['system']['name']:
                system['children'].append(asset)
        
print('--------------SYSTEMS----------------')
for system in systems:
    print('System: {}'.format(system['system']['name']))
    for child in system['children']:
        print('\tAsset: {}'.format(child['name']))

--------------SYSTEMS----------------
System: PXIChassis1
System: NI-cRIO-9068-190D5D5
System: RIO0
System: NI-cRIO-9068-190D673
System: RIO0
System: NI-cRIO-9068-190FDF5
System: RIO0
System: PXIChassis1
System: INTEL-NUC-2
	Asset: cDAQ9191-18B4B45Mod1
	Asset: Dev1
	Asset: cDAQ9191-18B4B45
	Asset: Dev2
System: INTEL-NUC-1
	Asset: cDAQ1
	Asset: cDAQ1Mod1
	Asset: myDAQ1
	Asset: cDAQ1Mod4
	Asset: cDAQ1Mod2
	Asset: cDAQ1Mod3
	Asset: GPIB0
	Asset: 
System: jprewitt5
	Asset: cDAQ1
	Asset: cDAQ1Mod1
	Asset: cDAQ1Mod3
	Asset: cDAQ1Mod2
	Asset: Asset 123
	Asset: NI-cRIO-9049-01234567
System: NI-cRIO-9049-01234567
	Asset: cRIO1
	Asset: Mod1
	Asset: Mod2
	Asset: Mod4
	Asset: Mod7
System: RIO0
System: sarlacc
System: Cirrostratus
System: Chassis 1
System: ASW-BGREY
System: DESKTOP-2I7QQ8F
System: PXIChassis1
System: VAT-REPO-VM
	Asset: Module XYZ
System: DESKTOP-2H5ROLH


### Generate asset utilization history

In [11]:
def shift_in_range(shift, start_range, end_range):
    shift_start = datetime(start_range.year, start_range.month, start_range.day, shift['start_hour'])
    shift_end = shift_start + timedelta(hours=8)
    if (shift_start >= start_range and shift_end <= end_range):
        return {'start': shift_start, 'end': shift_end}
    return None

for date_range in date_ranges:
    for shift in shifts:
        shift_range = shift_in_range(shift, date_range['start'], date_range['end'])
        if shift_range is not None:
            for operator_name in shift['operators']:
                low_utilization_percent = shift['operators'][operator_name]['low_utilization_percent']
                high_utilization_percent = shift['operators'][operator_name]['high_utilization_percent']
                random_utilization_percent = (random.random() * (high_utilization_percent-low_utilization_percent)) + low_utilization_percent
                for asset in shift['operators'][operator_name]['assets']:
                    minion_id, system_controller = get_system_controller_for_asset(asset, assets)
                    utilization_id = start_utilization(
                        system_controller=system_controller,
                        minion_id=minion_id,
                        assets=[asset],
                        utilization_category=random.choice(utilization_categories),
                        task_name=random.choice(task_names),
                        username=operator_name,
                        start_date=shift_range['start'].isoformat()+'Z'
                    )
                    random_utilization_shift_end = datetime(shift_range['start'].year, shift_range['start'].month, shift_range['start'].day, shift_range['start'].hour) + timedelta(seconds=((shift_range['end'] - shift_range['start']).total_seconds() * random_utilization_percent))
                    end_utilization(utilization_id, end_date=random_utilization_shift_end.isoformat()+'Z')